This notebook will load in the needed data from the milestoning simulation and save the cleaned data into a pair of tables for the milestoning coordinates and restraint parameters respectively.

If you are running this on google colab (or another cloud based site) you will need to clone the repository using the cell below (uncomment and run).

In [ ]:
#!git clone https://github.com/wesleymsmith/Milestoning_Analysis.git

In [2]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import tqdm
import copy
import gc
import sys
import os
import f90nml

import bokeh
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral11

from bokeh.models.mappers import CategoricalColorMapper

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import scipy as sp
from scipy.sparse import linalg

# Single Replica MD data

To start, lets load data from a sample MD milestoning simulation where in a single milestoning simulation was performed for milestone window.

We need to collect several pieces of information.

First, we need to collect simulation coordinates that are relevant to our milestoning.

In the case we are considering here, these were recorded in output data files generated when we applied the restraints needed for implementing our milestoning potentials.

On that note we also need to know the relevant restraint information that defined each milestone potential.
Specificaly we need to know where our 'flat bottom' (e.g. unrestrained or free simulation) portion lied for each window. This is available in the restraint parameter files from each simulation window.

In [2]:
dataDir='test_md_data'
#dataDir='Milestoning_analysis/test_md_data' #uncomment this line if you needed to clone the repo
dataFiles=np.sort(os.listdir(dataDir))
dataFiles

array(['window_00.rest', 'window_00_rest.dat', 'window_01.rest',
       'window_01_rest.dat', 'window_02.rest', 'window_02_rest.dat',
       'window_03.rest', 'window_03_rest.dat'], dtype='|S18')

The files ending in '.rest' are the restraint parameters for milestoning. They are formatted as fortran 90 namelists. The files ending in '.dat' are the ouput files for the simulation restraints and are in whitespace delimeted tables.

The column names for the '.dat' files are not given within the data file itself so we need to define them ourselves. The data file will contain some dummy columns which are present to make it more human readable.
We will label them accordingly so they can be easily removed later.

This simulation required a number of other restraints that are not relavent to the milestoning analysis. For instance, there are restraints to keep magnesium bound to key sites along the protein through which the ligand is being pulled so that the channel remains in its open state. For current purposes, we only need the Z coordinates for the waters-ligand restraints. They are labeled as 'W1L_Z' and 'W2L_Z'... More specifically, we only need 'W2L_Z' since this particular subset of the full milestoning data set only used water two to define its softwall restraints.

The cell below shows an example of what we will get upon reading in a '.dat' file

In [3]:
datColNames=np.concatenate([
     ['Frame'],
     ['MG%g'%iiMG for iiMG in np.arange(6)],
     ['DUMMYC_X','C_X','DUMMYC_Y','C_Y','DUMMYC_Z','C_Z','R1'],
     ['DUMMYW1L_X','W1L_X','DUMMYW1L_Y','W1L_Y','DUMMYW1L_Z','W1L_Z','W1L_R'],
     ['DUMMYW2L_X','W2L_X','DUMMYW2L_Y','W2L_Y','DUMMYW2L_Z','W2L_Z','W2L_R'],
    ])
pd.read_csv(dataDir+'/'+dataFiles[1],delim_whitespace=True,
            names=datColNames).head()

,Frame,MG0,MG1,MG2,MG3,MG4,MG5,DUMMYC_X,C_X,DUMMYC_Y,...,DUMMYW1L_Z,W1L_Z,W1L_R,DUMMYW2L_X,W2L_X,DUMMYW2L_Y,W2L_Y,DUMMYW2L_Z,W2L_Z,W2L_R
0,0,1.959,1.873,1.940,1.902,2.015,1.946,x:,-13.329,y:,...,z:,110.140,112.844,x:,10.766,y:,26.174,z:,-15.195,32.123
1,500,1.958,1.946,1.963,1.899,1.988,1.888,x:,-13.094,y:,...,z:,110.548,113.265,x:,10.342,y:,26.464,z:,-14.744,32.010
2,1000,1.935,2.071,1.951,1.895,1.986,1.950,x:,-13.316,y:,...,z:,110.709,113.389,x:,10.483,y:,26.262,z:,-14.688,31.864
3,1500,1.969,1.922,1.990,1.912,1.984,1.942,x:,-12.631,y:,...,z:,110.696,113.418,x:,10.123,y:,26.661,z:,-14.631,32.052
4,2000,1.883,1.915,1.894,1.895,2.094,1.984,x:,-12.681,y:,...,z:,111.143,113.829,x:,10.083,y:,26.527,z:,-14.230,31.746


In addition to the reaction coordinate data from the '.dat' files, we also need to know about the parameters of the restraints that were used in each window. As mentioned before, this is found in the '.rest' files. These files contain only soft wall restraint settings.

These fortran 90 namelist formatted files can be loaded using the 'f90nml' package. We show an example in the cell below.

The relevant parameters are 'r2' and 'r3', which define the flat bottom portion of the milestoning restraint, and 'rk2' and 'rk3' which define the force constant of the pseudo-harmonic walls.

In [4]:
f90nml.read(dataDir+'/'+dataFiles[0])

Namelist([('rst',
           [Namelist([('iat', [-1, -1]),
                      ('r1', 95.0),
                      ('r2', 109.0),
                      ('r3', 111.0),
                      ('r4', 135.0),
                      ('rk2', 0.0),
                      ('rk3', 0.0),
                      ('iresid', 0),
                      ('fxyz', [0, 0, 1]),
                      ('outxyz', 1),
                      ('igr1', 102701),
                      ('igr2', 22402)]),
            Namelist([('iat', [-1, -1]),
                      ('r1', 5.0),
                      ('r2', 14.0),
                      ('r3', 16.0),
                      ('r4', 40.0),
                      ('rk2', 100.0),
                      ('rk3', 100.0),
                      ('iresid', 0),
                      ('fxyz', [0, 0, 1]),
                      ('outxyz', 1),
                      ('igr1', 136412),
                      ('igr2', 22402)])])])

We are now ready to start loading in our data. In doing so, we will need to keep track of which window each data file came from.

We will produce two combined tables. One for the '.dat' files and another for the '.rest' files.

In [5]:
datFiles=[datFile for datFile in dataFiles if '.dat' in datFile]
restFiles=[restFile for restFile in dataFiles if '.rest' in restFile]
print 'reaction coordinate data files:',
print datFiles
print 'soft-wall restraint parameter files:',
print restFiles

windowDataTables=[]

restParmNMLname='rst'
restParmNames=['r2','r3','rk2','rk3']
restParmSubsetInds=[0,1]
parmData={}
parmData['Window']=[]
for iParm,parmInd in enumerate(restParmSubsetInds):
    for parmName in restParmNames:
        parmData['W%gL_'%(iParm+1)+parmName]=[]

for datFile in datFiles:
    print 'loading milestone coordinate data from %s'%datFile
    datFilePath=dataDir+'/'+datFile
    window=np.int(datFile.split('_')[1])
    tempTable=pd.read_csv(datFilePath,delim_whitespace=True,
                          names=datColNames)
    tempTable['Window']=window
    windowTable=tempTable[['Window']].copy()
    windowTable['Time']=tempTable['Frame']
    windowTable['X']=tempTable['W2L_Z'].abs()
    windowDataTables.append(windowTable.copy())
simData=pd.concat(windowDataTables)
print simData.head()    
simData.to_csv("Simulation_Milestone_Coordinate_Data.csv",index=False)

for restFile in restFiles:
    print 'loading milestone restraint data from %s'%restFile
    restFilePath=dataDir+'/'+restFile
    window=np.int(restFile.split('.')[0].split('_')[-1])
    tempNMLdata=f90nml.read(restFilePath)
    parmData['Window'].append(window)
    for restParmInd in restParmSubsetInds:
        for restParmName in restParmNames:
            parmData['W%gL_'%(restParmInd+1)+restParmName].append(tempNMLdata[
                restParmNMLname][restParmInd][restParmName])
windowRestraintTable=pd.DataFrame(parmData)
print windowRestraintTable
windowRestraintTable.to_csv("Simulation_Milestone_Restraint_Data.csv",index=False)

reaction coordinate data files: ['window_00_rest.dat', 'window_01_rest.dat', 'window_02_rest.dat', 'window_03_rest.dat']
soft-wall restraint parameter files: ['window_00.rest', 'window_01.rest', 'window_02.rest', 'window_03.rest']
loading milestone coordinate data from window_00_rest.dat
loading milestone coordinate data from window_01_rest.dat
loading milestone coordinate data from window_02_rest.dat
loading milestone coordinate data from window_03_rest.dat
   Window  Time       X
0       0     0  15.195
1       0   500  14.744
2       0  1000  14.688
3       0  1500  14.631
4       0  2000  14.230
loading milestone restraint data from window_00.rest
loading milestone restraint data from window_01.rest
loading milestone restraint data from window_02.rest
loading milestone restraint data from window_03.rest
   W1L_r2  W1L_r3  W1L_rk2  W1L_rk3  W2L_r2  W2L_r3  W2L_rk2  W2L_rk3  Window
0   109.0   111.0      0.0      0.0    14.0    16.0    100.0    100.0       0
1   107.0   109.0      0.

# Mulit-Replica MD milestoning simulation

Next, lets consider the case where some milestoning windows needed to be run in replicate. E.g. for some windows there are multiple individual replicates of the milestoning simulation that have been run (with different starting conformation / random seeds). This is useful, for instance, when it is found that too few edge crossings have occured over a given window edge and addtional replicates are run to increase sampling. Alternatively, this could be done to take advantage of having multiple gpus, workstations, cluster nodes, etc. wherein one would like to increase sampling by running multiple replicates rather than a single long simulation. I.e. cases where running additional simulations is more efficient than using additional resources on a single simulation.

In any case, care needs to be taken when collecting and analyzing milestoning data here because the individual replicas are not contiguous simulations but rather mimic an ensemble of simulations. For instane, if we treated replica simulations as contiguous, we could end up over counting / miscounting transition times where one replica ends and another starts. The solution is to analyze each replica individually and then agglomerate the results. Again, care must be taken here because we need to normalize over the total time spent in a given window, so we cannot simply take a raw mean of values over each replica. Thus, it is important to keep track of what data is coming from what replica in addition to each window.

For now, this simply means we need to add a column to our data to track replicas.

Since each replicate uses the same restraint setup, we need only modify how we store the simulation coordinates that are loaded.

In [8]:
dataDir='test_multiReplica_md_data/' #set this to the appropriate directory name
#dataDir='Milestoning_analysis/test_md_data' #uncomment this line if you needed to clone the repo
dataFiles=np.sort(os.listdir(dataDir))
dataFiles

array(['window_00.rest', 'window_00_rest.dat', 'window_00_rest.rep_1.dat',
       'window_00_rest.rep_2.dat', 'window_00_rest.rep_3.dat',
       'window_00_rest.rep_4.dat', 'window_00_rest.rep_5.dat',
       'window_00_rest.rep_6.dat', 'window_00_rest.step_1.dat',
       'window_01.rest', 'window_01_rest.dat',
       'window_01_rest.step_1.dat', 'window_01_rest.step_2.dat',
       'window_02.rest', 'window_02_rest.dat',
       'window_02_rest.step_1.dat', 'window_03.rest',
       'window_03_rest.dat', 'window_03_rest.step_1.dat',
       'window_04.rest', 'window_04_rest.dat',
       'window_04_rest.step_1.dat'], dtype='|S25')

The files ending in '.rest' are the restraint parameters for milestoning. They are formatted as fortran 90 namelists. The files ending in '.dat' are the ouput files for the simulation restraints and are in whitespace delimeted tables.

The column names for the '.dat' files are not given within the data file itself so we need to define them ourselves. The data file will contain some dummy columns which are present to make it more human readable.
We will label them accordingly so they can be easily removed later.

In addition to the reaction coordinate data from the '.dat' files, we also need to know about the parameters of the restraints that were used in each window. As mentioned before, this is found in the '.rest' files. These files contain only soft wall restraint settings.

These fortran 90 namelist formatted files can be loaded using the 'f90nml' package. We show an example in the cell below.

The relevant parameters are 'r2' and 'r3', which define the flat bottom portion of the milestoning restraint, and 'rk2' and 'rk3' which define the force constant of the pseudo-harmonic walls.

The data presented in the .dat files will correspond to the entries in the .rest files.

One important detail here is that each entry in the .rest file will tell us important information about how the data in the .dat file will be formatted.

The nmr restraint format used here is a bit too complex to go into fine detail. Interested readers should look in the appropriate section of the AMBER modeling and simulation manual.

In the case of more advanced restraint files, we would need to examine the 'iat' entry for each namelist entry to determine what kind of restraint we have.
For instance if there are 2 entries, we have a distance restraint. Angle type entries would have 3 iat entries and dihedral type entries would have 4. There can even be more than 4 entries in iat for various specialized restraint types...

For our purposes here, we only need to consider distance type restraints. In particular, our milestone windows are defined as 'flat bottom' harmonic potentials. The 'flat bottom' of these wells is what is important to us as that defines the region of our milestone window we need to sample from.

As such we can focus on a couple entry types. 

First, the entries r1 through r4 tell us about points of interest for our flat bottom potential. In particular, we want to know r2 and r3 which define the 'flat bottom' region.

Second, we need to look for the 'outxyz' flag. This is not so important for defining the restraint itself, but it will tell us important inforamtion about how our .dat file will be formatted. When set to 0 or absent, a single value corresponding to the measured distance will be output. When set to 1, additional information will be written detailing the x, y, and z coordinates of the restraint vector along with the total distance. Moreover, the data will contain 'x:', 'y:', and 'z:' in the columns preceding the x, y, and z data respectively. We will need to include appropriate dummy columns as placeholders.

Lets have a look at the first such file

In [19]:
restFiles=[dataFile for dataFile in dataFiles if '.rest' in dataFile]
tempRestFile=f90nml.read(dataDir+'/'+restFiles[0])
tempRestFile

Namelist([('rst',
           [Namelist([('iat', [65842, 65843]),
                      ('r1', 0),
                      ('r2', 0),
                      ('r3', 2.5),
                      ('r4', 3.88),
                      ('rk2', 10.0),
                      ('rk3', 10.0)]),
            Namelist([('iat', [65842, 65846]),
                      ('r1', 0),
                      ('r2', 0),
                      ('r3', 2.5),
                      ('r4', 3.88),
                      ('rk2', 10.0),
                      ('rk3', 10.0)]),
            Namelist([('iat', [65842, 65849]),
                      ('r1', 0),
                      ('r2', 0),
                      ('r3', 2.5),
                      ('r4', 3.88),
                      ('rk2', 10.0),
                      ('rk3', 10.0)]),
            Namelist([('iat', [65842, 65852]),
                      ('r1', 0),
                      ('r2', 0),
                      ('r3', 2.5),
                      ('r4', 3.88),
                  

Now we need to loop over each entry in this restraint parameter file and determine what the corresponding 'outxyz' entry is set to. If not explicitly present it can be assumed to be set to its default value of 1.

In [21]:
mainNamelistType='rst'
restXYZ=[rstDat['outxyz'] if 'outxyz' in rstDat else 0 for rstDat in tempRestFile[mainNamelistType]]
restXYZ

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

This simulation required a number of other restraints that are not relavent to the milestoning analysis. For instance, there are restraints to keep magnesium bound to key sites along the protein through which the ligand is being pulled so that the channel remains in its open state. For current purposes, we only need the Z coordinates for the waters-ligand restraints. They are labeled as 'W1L_Z' and 'W2L_Z'... More specifically, we only need 'W2L_Z' since this particular subset of the full milestoning data set only used water two to define its softwall restraints.

The cell below shows an example of what we will get upon reading in a '.dat' file

Previously we defined the needed columns by hand. Since there quite a few entries to track now, we need to automate this a bit.

In [24]:
datColNames=['Frame']
nRest=len(restXYZ)
nDigits=int(np.ceil(np.log10(nRest)))
baseNameStr='%s0%g%s'%('Rst%',nDigits,'g')
for iRst,outxyzVal in enumerate(restXYZ):
    if outxyzVal==0:
        datColNames=np.concatenate([
            datColNames,
            [baseNameStr%iRst]])
    else:
        datColNames=np.concatenate([
            datColNames,
            [(baseNameStr%iRst)+('_%s'%crd) for crd in [
                'DummyX','X','DummyY','Y','DummyZ','Z','R']]])
pd.read_csv(dataDir+'/'+dataFiles[1],delim_whitespace=True,
            names=datColNames).head()

,Frame,Rst00,Rst01,Rst02,Rst03,Rst04,Rst05,Rst06_DummyX,Rst06_X,Rst06_DummyY,...,Rst10_DummyZ,Rst10_Z,Rst10_R,Rst11_DummyX,Rst11_X,Rst11_DummyY,Rst11_Y,Rst11_DummyZ,Rst11_Z,Rst11_R
0,0,1.942,1.972,1.884,1.977,1.957,1.878,x:,0.415,y:,...,z:,52.096,53.414,x:,10.588,y:,5.191,z:,52.096,53.414
1,500,1.953,1.870,1.948,1.991,1.994,1.922,x:,0.054,y:,...,z:,52.108,53.427,x:,10.595,y:,5.196,z:,52.108,53.427
2,1000,2.093,1.906,1.994,1.840,1.891,2.043,x:,0.710,y:,...,z:,52.104,53.423,x:,10.583,y:,5.217,z:,52.104,53.423
3,1500,2.017,1.952,1.952,2.007,1.869,1.930,x:,0.353,y:,...,z:,52.100,53.420,x:,10.595,y:,5.199,z:,52.100,53.420
4,2000,1.936,1.936,1.877,2.023,1.919,1.941,x:,0.459,y:,...,z:,52.095,53.415,x:,10.594,y:,5.199,z:,52.095,53.415


We are now ready to start loading in our data. In doing so, we will need to keep track of which window and replica each data file came from.

We will produce two combined tables. One for the '.dat' files and another for the '.rest' files.

Since our coordinate and restraint data contains a lot of uneeded information, we will use the 'restParmSubsetInds' variable to define a list of which restraint entries are needed from the restraint data files.
Additionally, we only need to know the values of certain sub-entries from each restraint.
Specifically we need 'r2' and 'r3' to locate the bounds of our milestone window. Additionally, we may want to know the force constant information at some point so we will retain 'rk2' and 'rk3' as well.

Similarly, we only need certain data columns from coordinate data as well.
These can be specified using the 'crdColumns' variable.

In [42]:
datFiles=[datFile for datFile in dataFiles if '.dat' in datFile]
restFiles=[restFile for restFile in dataFiles if '.rest' in restFile]
print 'reaction coordinate data files:',
print datFiles
print 'soft-wall restraint parameter files:',
print restFiles

windowDataTables=[]

restParmNMLname='rst' #This is the main namelist... probably don't need to change it

### MODIFY THESE VARIABLES AS NEEDED ### ###
#The next 3 variables should be changed to match the restraint and coordinate data
#setup you have
restParmNames=['r2','r3','rk2','rk3'] #select needed restraint sub-entry names
restParmSubsetInds=[7,8] #Select the needed restraint entries
crdColumns=['Rst07_Z','Rst08_Z'] #Select the needed columns from the coordinate data
### ### ### ### ### ### ### ### ### ### ###

parmData={}
parmData['Window']=[]
for iParm,parmInd in enumerate(restParmSubsetInds):
    for parmName in restParmNames:
        parmData[baseNameStr%(parmInd)+'_%s'%parmName]=[]
        
for restFile in restFiles:
    print 'loading milestone restraint data from %s'%restFile
    restFilePath=dataDir+'/'+restFile
    window=np.int(restFile.split('.')[0].split('_')[-1])
    tempNMLdata=f90nml.read(restFilePath)
    parmData['Window'].append(window)
    for iParm,restParmInd in enumerate(restParmSubsetInds):
        for restParmName in restParmNames:
            parmData[baseNameStr%(restParmInd)+'_%s'%restParmName].append(tempNMLdata[
                restParmNMLname][restParmInd][restParmName])
windowRestraintTable=pd.DataFrame(parmData)
windowRestraintTable=windowRestraintTable[np.concatenate([
        ['Window'],
        [colName for colName in np.sort(parmData.keys()) \
         if colName!='Window']])]
print windowRestraintTable
windowRestraintTable.to_csv("Simulation_Milestone_Restraint_Data.MultiRep.csv",index=False)

for datFile in datFiles:
    print 'loading milestone coordinate data from %s'%datFile
    datFilePath=dataDir+'/'+datFile
    window=np.int(datFile.split('_')[1])
    if 'rep_' in datFile:
        rep=np.int(datFile.split('.')[1].split('_')[1])
    else:
        rep=1
    tempTable=pd.read_csv(datFilePath,delim_whitespace=True,
                          names=datColNames)
    tempTable['Window']=window
    tempTable['Rep']=rep
    windowTable=tempTable[['Window','Rep']].copy()
    windowTable['Time']=tempTable['Frame']
    for crdColumn in crdColumns:
        windowTable[crdColumn]=tempTable[crdColumn].abs()
    windowDataTables.append(windowTable.copy())
simData=pd.concat(windowDataTables)
print simData.head()    
simData.to_csv("Simulation_Milestone_Coordinate_Data.MultiRep.csv",index=False)

reaction coordinate data files: ['window_00_rest.dat', 'window_00_rest.rep_1.dat', 'window_00_rest.rep_2.dat', 'window_00_rest.rep_3.dat', 'window_00_rest.rep_4.dat', 'window_00_rest.rep_5.dat', 'window_00_rest.rep_6.dat', 'window_00_rest.step_1.dat', 'window_01_rest.dat', 'window_01_rest.step_1.dat', 'window_01_rest.step_2.dat', 'window_02_rest.dat', 'window_02_rest.step_1.dat', 'window_03_rest.dat', 'window_03_rest.step_1.dat', 'window_04_rest.dat', 'window_04_rest.step_1.dat']
soft-wall restraint parameter files: ['window_00.rest', 'window_01.rest', 'window_02.rest', 'window_03.rest', 'window_04.rest']
loading milestone restraint data from window_00.rest
loading milestone restraint data from window_01.rest
loading milestone restraint data from window_02.rest
loading milestone restraint data from window_03.rest
loading milestone restraint data from window_04.rest
   Window  Rst07_r2  Rst07_r3  Rst07_rk2  Rst07_rk3  Rst08_r2  Rst08_r3  \
0       0     111.5     113.5        0.0       

Our simulation data is a bit more complex this time around since we have more replicas.

Also, as with the previous data set we have multiple coordinates tracked.
It would be nice to be able to automatically detect which coordinates we need to use.

We do so here by making use of the k values for each window. Any restraint which has a k-value of zero can essentially be ignored.

To make life a little easier on the analysis step to come, lets bin the coordinate data into the appropriate milestone windows here.

In order to do this, lets first extract the relevant window data from the restraint table.

One point of interest here is that the K values for each window will be important since they will tell us which coordinates were relevant for each window. This will become important shortly

In [44]:
binMins=np.array(windowRestraintTable[[
    colName for colName in windowRestraintTable.columns \
    if 'r2' in colName]])
binMaxs=np.array(windowRestraintTable[[
    colName for colName in windowRestraintTable.columns \
    if 'r3' in colName]])
binCenters=(binMins+binMaxs)/2
binRK2s=np.array(windowRestraintTable[[
    colName for colName in windowRestraintTable.columns \
    if 'rk2' in colName]])
binRK3s=np.array(windowRestraintTable[[
    colName for colName in windowRestraintTable.columns \
    if 'rk3' in colName]])
binKs=np.sqrt(binRK2s*binRK3s)
print 'binMins:',
print binMins
print 'binMaxs:',
print binMaxs
print 'binCenters:',
print binCenters
print 'binKs:'
print binKs

binMins: [[111.5   4.8]
 [109.5   6.8]
 [107.5   8.8]
 [105.5  10.8]
 [103.5  12.8]]
binMaxs: [[113.5   6.8]
 [111.5   8.8]
 [109.5  10.8]
 [107.5  12.8]
 [105.5  14.8]]
binCenters: [[112.5   5.8]
 [110.5   7.8]
 [108.5   9.8]
 [106.5  11.8]
 [104.5  13.8]]
binKs:
[[  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]
 [  0. 100.]]


Now we need to create a function that can assigne a given coordinate data point to the appropriate milestone window.

To do this, we need to know
    1. which window the coordinate came frome
    2. the bin centers for each window
    3. the k-values for each window
    4. the data coordinates
    
The first step is to select which restraint coordinates are relevant. This amounts to finding which restraint coordinates had non-zero K values.

From this we then subset our data coordinates and window centers, keeping only coordinates for which the restraint had a non-zero K value.

Finally we compute the distance from the remaining data coordinates using the corresponding coordinates of each window center. We then return the index of the window with the center that was closest to the data point.

In [104]:
#Function to compute bin index based on window and coordinate data
#X contains the coordinate data columns from the coordinate data table along
#with the 'Window' column
#e.g. x[0]=window, x[1] through x[n+1] are coordinates 0 through n
def binDataFunction(x,centers,kVals,verbose=False):
    X=np.array(x)
    if verbose:
        print 'x',
        print X
    coordInds=kVals[int(X[0])]>0 #only consider restraints with non-zero K values
    coordSet=X[1:][coordInds] #get subset of relevant data point coordinates
    centerSet=centers[:,coordInds] #get subset of relevant window center coordinates
    if verbose:
        print 'coordInds',
        print coordInds
        print 'coordSet',
        print coordSet
        print 'centerSet',
        print centerSet
        print 'outDists',
        print np.sqrt(np.sum(1.0*(centerSet-coordSet)**2,axis=1))
    #return the bin index of the center that is closest to the data point given
    return np.argmin(np.sqrt(np.sum(1.0*(centerSet-coordSet)**2,axis=1)))

In [105]:
#lets test the function
binDataFunction(np.array(simData[['Window','Rst07_Z','Rst08_Z']].iloc[31000]),
                binCenters,binKs,verbose=True)

x [  1.    110.489   7.832]
coordInds [False  True]
coordSet [7.832]
centerSet [[ 5.8]
 [ 7.8]
 [ 9.8]
 [11.8]
 [13.8]]
outDists [2.032 0.032 1.968 3.968 5.968]


1

We are now ready to bin our simulation restraint coordinate data.

We will need the 'Window' column along with all columns that had 'Rst' in the column name.

We will feed the window centers and k values we constructed above using a keyword dictionary which we will supply to the pandas 'apply' command.

In [107]:
binKwds={
    'centers':binCenters,
    'kVals':binKs,
    #'verbose':True
}
simData['X_Index']=simData[np.concatenate([
    ['Window'],
    [colName for colName in simData.columns if 'Rst' in colName]
])].apply(binDataFunction,axis=1,**binKwds)
simData.head()

,Window,Rep,Time,Rst07_Z,Rst08_Z,X_Index
0,0,1,0,112.312,5.986,0
1,0,1,500,112.322,5.961,0
2,0,1,1000,112.426,5.891,0
3,0,1,1500,112.312,5.974,0
4,0,1,2000,112.004,6.272,0


As a quick check, lets make sure we see transitions between adjacent windows.

We can do this using the pandas groupby-aggregate method and serializing the list of unique X_Index values seen for each window

In [111]:
simData.groupby(['Window']).agg({'X_Index':lambda x: ','.join([str(val) for val in np.unique(x)])})

,X_Index
Window,
0,"0,1"
1,"0,1,2"
2,"1,2,3"
3,"2,3,4"
4,"3,4"


Now lets re-export this data so that we can have the X_Index available during analysis.

In [112]:
simData.to_csv("Simulation_Milestone_Coordinate_Data.MultiRep.csv",index=False)